Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [0]:
# I have chosen the nfl play by play for the years 2017-2019 as the data set.
# I have chosen as my target (my Y) to be if the play is a pass play  or a run play.  That
# is I want to predict the probability of the next play being a pass or a run.
# At the moment there are two columns that have if the play was a pass or a run.
# I will combine them into one column.  

# I am going to choose to most likely use a classification type model.  AT the moment 
# I don't know how large the majority class is, becuase I still need to do some feature 
# engineering to find out the amounts, but I don't believe that either the pass play or the 
# run play will be larger than 70% for the data set.

# I will train on the year 2017 and 2018 and test with the year 2019.  If I need I can grab another
# year ie.,  2016 to make the train/validate set even larger.